In [69]:
%%capture --no-stderr
! pip install -U langchain_community tiktoken langchain-openai langchain-cohere langchainhub chromadb langchain langgraph  tavily-python --quiet

In [70]:
!pip install langchain_google_genai --quiet

In [71]:
!pip install langchain_groq --quiet

In [72]:
import os
from google.colab import userdata
tavily_api_key = userdata.get("TAVILY_API_KEY")
groq_api_key = userdata.get("GROQ_API_KEY")
cohere_api_key = userdata.get("COHERE_API_KEY")
# Use the actual API key here:
os.environ["LANGCHAIN_TRACING_V2"] = userdata.get("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
langchain_api_key =userdata.get("LANGSMITH_API_KEY")

In [73]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))

In [74]:
import warnings
warnings.filterwarnings("ignore")

In [75]:
google_api_key = userdata.get("GOOGLE_API_KEY")

embd = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=google_api_key
)

In [76]:
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)


In [77]:
print(len(doc_splits))

88


In [78]:
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=embd,
)
retriever = vectorstore.as_retriever()

In [79]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    groq_api_key = groq_api_key
)
print("model retrieved")

model retrieved


In [107]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Literal
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["vectorstore", "web_search"] = Field(
        ...,
        description="Given a user question choose to route it to web search or a vectorstore.",
    )
structured_llm_router = llm.with_structured_output(RouteQuery)

# Prompt
system = """You are an expert at routing a user question to a vectorstore or web search.
The vectorstore contains documents related to agents, prompt engineering, and adversarial attacks.
Use the vectorstore for questions on these topics. Otherwise, use web-search."""
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | structured_llm_router
print(
    question_router.invoke(
        {"question": "Who will the Bears draft first in the NFL draft?"}
    )
)
print(question_router.invoke({"question": "What are the types of agent memory?"}))

datasource='web_search'
datasource='vectorstore'


In [83]:
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )
llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    groq_api_key = groq_api_key
)
structured_llm_grader = llm.with_structured_output(GradeDocuments)
system = """You are a grader assessing relevance of a retrieved document to a user question. \n
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)
retrieval_grader = grade_prompt | structured_llm_grader
question = "agent memory"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

binary_score='no'


In [84]:
os.environ["LANGCHAIN_TRACING_V2"] = langchain_api_key

In [85]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = hub.pull("rlm/rag-prompt" , api_key = langchain_api_key)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

In the context of LLM-powered autonomous agents, memory refers to several types of information storage and retrieval processes. Sensory memory represents raw inputs like text or images, short-term memory handles in-context learning within a finite context window, and long-term memory is an external vector store for information retrieval. To optimize long-term memory retrieval, maximum inner product search (MIPS) and approximate nearest neighbors (ANN) algorithms are often used.


In [86]:
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )
llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    groq_api_key = groq_api_key
)
structured_llm_grader = llm.with_structured_output(GradeHallucinations)
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n
     Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)
hallucination_grader = hallucination_prompt | structured_llm_grader
hallucination_grader.invoke({"documents": docs, "generation": generation})

GradeHallucinations(binary_score='yes')

In [87]:
print("hello")

hello


In [88]:
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )


# LLM with function call
llm =  ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    groq_api_key = groq_api_key
)
structured_llm_grader = llm.with_structured_output(GradeAnswer)

# Prompt
system = """You are a grader assessing whether an answer addresses / resolves a question \n
     Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_grader = answer_prompt | structured_llm_grader
answer_grader.invoke({"question": question, "generation": generation})

GradeAnswer(binary_score='yes')

In [89]:
# Prompt
system = """You a question re-writer that converts an input question to a better version that is optimized \n
     for vectorstore retrieval. Look at the input and try to reason about the underlying semantic intent / meaning."""
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question. Just give me the question",
        ),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

'"How can I access or inquire about the information stored in the memory of a language model agent?"'

In [90]:
import os

os.environ["TAVILY_API_KEY"] = tavily_api_key

from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool  = TavilySearchResults(k=3)

# GRAPH
capturing the flow of the graph

## GRAPH STATE

In [91]:
from typing import List
from typing_extensions import TypedDict

class GraphState(TypedDict):
  question : str
  generation: str
  documents : List[str]

## GRAPH FLOW

In [92]:
from langchain.schema import Document
def retrieve(state):
  print("--RETRIEVING--")
  question = state["question"]
  documents = retriever.invoke(question)
  return {"documents": documents ,"question":question}

In [93]:
def generate(state):
  print("--GENERATE--")
  question = state["question"]
  documents = state["documents"]

  generation = rag_chain.invoke({"context":documents , "question":question})
  return {"documents": documents , "question":question , "generation":generation}

In [94]:
def grade_document(state):
  print("----CHECKING DOCUMENT RELEVANCE----")
  documents = state["documents"]
  relevant_docs = []
  question = state["question"]

  for d in documents :
    score = retrieval_grader.invoke({"question": question, "document": d.page_content})
    grade = score.binary_score
    if grade == "yes":
        print("---GRADE: DOCUMENT RELEVANT---")
        relevant_docs.append(d)
    else:
        print("---GRADE: DOCUMENT NOT RELEVANT---")
        continue
  return {"documents": relevant_docs, "question": question}


In [95]:
def transform_query(state):
  print("--TRANSFORMING THE QUERY--")
  question = state["question"]
  documents = state["documents"]
  better_question = question_rewriter.invoke({"question":question})
  return {"documents": documents, "question": better_question}

In [96]:
def web_search(state):
  print("-- DOING WEB SEARCH --")
  question = state["question"]
  docs = web_search_tool.invoke({"query": question})
  web_results = "\n".join([d["content"] for d in docs])
  web_results = Document(page_content = web_results)
  return {"documents": web_results, "question": question}

In [97]:
def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    question = state["question"]
    source = question_router.invoke({"question": question})
    if source.datasource == "web_search":
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "web_search"
    elif source.datasource == "vectorstore":
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"

In [98]:
def decide_to_generate(state):
  """
  decides to whether to generate the answer or to re-generate the question
  """
  print("--ASSESSING THE RELEVANT DOCUMENTS --")
  question = state["question"]
  relevant_docs = state["documents"]
  if not relevant_docs :
    print("--DECISION : ALL THE DOCUMENT FOUND ARE NOT RELEVANT TO THE QUERY , SO WE NEED TO TRANSFORM IT ")
    return "transform_query"
  else :
    print("--DECISION : GENERATE THE ANSWER")
    return "generate"

In [104]:
def grade_generation_v_documents_and_question(state):
  print("--CHECKING THE HALLUCINATIONS--")
  question = state["question"]
  generation = state["generation"]
  documents = state["documents"]

  score = hallucination_grader.invoke({"documents": documents , "generation": generation})
  grade = score.binary_score
  if grade == "yes":
    print("---GENERATION IS GROUNDED IN THE DOCUMENTS--- ")
    print("--GRADING GENERATION VS QUESTION--")
    score = answer_grader.invoke({"question" :question , "generation" : generation})
    grade = score.binary_score
    if grade == "yes":
      print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
      # Check question-answering
      print("---GRADE GENERATION vs QUESTION---")
      score = answer_grader.invoke({"question": question, "generation": generation})
      grade = score.binary_score
      if grade == "yes":
          print("---DECISION: GENERATION ADDRESSES QUESTION---")
          return "useful"
      else:
          print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
          return "not useful"
    else:
      print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
      return "not supported"
    return ""

## BUILDING THE GRAPH

In [102]:
from langgraph.graph import END , StateGraph , START
workflow = StateGraph(GraphState)

#defining the nodes

#defining the nodes
workflow.add_node("web_search" , web_search)
workflow.add_node("retrieve", retrieve)
workflow.add_node("grade_documents" , grade_document)
workflow.add_node("generate", generate)
workflow.add_node("transform_query", transform_query)

#building the graph
workflow.add_conditional_edges(
    START,
    question_router,
    {
        "web_search": "web_search",
        "vectorstore": "retrieve",
    },
)
workflow.add_edge("web_search", "generate")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "retrieve")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "transform_query",
    },
)

# Compile
app = workflow.compile()


In [108]:
from pprint import pprint
inputs = {
    "question": "What player at the Bears expected to draft first in the 2024 NFL draft?"
}

for output in app.invoke(inputs):
    for key, value in output.items():
        pprint(f"Node '{key}':")
        # Ensure 'generation' key exists
        if "generation" in value:
            pprint(value["generation"])
        else:
            pprint("No generation found at this step.")
    pprint("\n---\n")


TypeError: unhashable type: 'RouteQuery'